In [8]:
import asyncio
import traceback
import sounddevice as sd
from scipy.io.wavfile import write
import base64

from hume import HumeStreamClient
from hume.models.config import BurstConfig, ProsodyConfig

# Record audio
samplerate = 44100  # Hertz
duration = 10  # seconds
filename = 'output.wav'
print(f"Recording for {duration} seconds...")
myrecording = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=2, blocking=True)
print("Recording complete. Saving the audio as output.wav")
write(filename, samplerate, myrecording)

# Function to generate audio stream (you may need to adjust this to fit your needs)
def generate_audio_stream(filename, chunk_size=1024):
    with open(filename, "rb") as f:
        while (chunk := f.read(chunk_size)):
            yield chunk

# Function to encode audio (base64 encoding)
def encode_audio(audio_sample):
    return base64.b64encode(audio_sample)

# Hume API interaction
async def main():
    try:
        client = HumeStreamClient("1Fuo6eVLpIj6ndhmC5VXllArH67eOcaSA0XLX3sHdU2SdEy5")
        burst_config = BurstConfig()
        prosody_config = ProsodyConfig()
        async with client.connect([burst_config, prosody_config]) as socket:
            for sample_number, audio_sample in enumerate(generate_audio_stream(filename)):
                encoded_sample = encode_audio(audio_sample)
                await socket.reset_stream()
                result = await socket.send_bytes(encoded_sample)
                print(f"\nStreaming sample {sample_number + 1}")
                # Add here your code to handle the 'result' data
    except Exception:
        print(traceback.format_exc())

loop = asyncio.get_event_loop()
loop.create_task(main())


Recording for 10 seconds...
Recording complete. Saving the audio as output.wav


<Task pending name='Task-12' coro=<main() running at /tmp/ipykernel_9740/3007795248.py:30>>


Streaming sample 1
Traceback (most recent call last):
  File "/tmp/ipykernel_9740/3007795248.py", line 39, in main
    result = await socket.send_bytes(encoded_sample)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/emil/anaconda3/envs/moodring/lib/python3.11/site-packages/hume/_stream/stream_socket.py", line 80, in send_bytes
    return await self._send_bytes_str(bytes_str)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/emil/anaconda3/envs/moodring/lib/python3.11/site-packages/hume/_stream/stream_socket.py", line 179, in _send_bytes_str
    return await self._send_payload(payload)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/emil/anaconda3/envs/moodring/lib/python3.11/site-packages/hume/_stream/stream_socket.py", line 196, in _send_payload
    raise HumeClientException.from_error(code, error)
hume.error.hume_client_exception.HumeClientException: hume(E0200): Failed to parse data as a valid media file. Please check that you have properl